In [1]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pandas as pd 
import numpy as np 
import pickle

In [2]:
## Load trained model, pickle files
model = load_model('model.h5')

# load encoders and scalar
with open ('ohe_geo.pkl' , 'rb') as file:
    ohe_geo = pickle.load(file)
    
with open ('label_encoder_gender.pkl' , 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open ('scaler.pkl' , 'rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France' ,
    'Gender' : 'Male' ,
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [4]:
## OHE Geography
geo_encoded = ohe_geo.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray() , columns = ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/opt/anaconda3/envs/dl_venv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data])

In [6]:
input_df = pd.concat([input_df.reset_index(drop = True), geo_encoded_df], axis = 1)
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
# Encode categorical variable
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df.drop('Geography' , axis = 1 , inplace= True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## Scaling the data
input_scaled = scaler.transform(input_df)
input_scaled

AttributeError: 'tuple' object has no attribute 'transform'

In [ ]:
# predict churn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 28ms/step


array([[0.05563342]], dtype=float32)

In [ ]:
prediction_prob =  prediction[0][0]
prediction_prob

0.05563342

In [ ]:
if prediction_prob > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
